In [23]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import nltk
import string

from sklearn.model_selection import learning_curve


from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import matplotlib.pyplot as plt

## Read and Prep Data


In [2]:
tqdm.pandas()

In [3]:
# from google.colab import drive
# drive.mount('/content/gdrive') 
# path = '/content/gdrive/MyDrive/IFT 6390/kaggle2/'

In [6]:
# input_path = path+'data/input/'
input_path = 'data/input/'

In [7]:
all_train_data = pd.read_csv(input_path + 'train_data_prepped.csv').fillna('')
all_test_data = pd.read_csv(input_path + 'test_data_prepped.csv').fillna('')

In [8]:
all_train_data.head(1)

,id,text,cat,text_no_punc,text_no_numerals,text_no_sw,text_porter_stemmed,text_lancaster_stemmed,text_lemmatized
0,0,anyway im getting of for a while,train,anyway im getting of for a while,anyway im getting of for a while,anyway im getting,anyway im get of for a while,anyway im get of for a whil,anyway im getting of for a while


In [9]:
all_test_data.head(1)

,id,text,cat,text_no_punc,text_no_numerals,text_no_sw,text_porter_stemmed,text_lancaster_stemmed,text_lemmatized
0,0,so cut... missed out on all the tce extreme me...,test,so cut missed out on all the tce extreme merch...,so cut missed out on all the tce extreme merch...,cut missed tce extreme merch! sizes,so cut miss out on all the tce extrem merch! i...,so cut miss out on al the tce extrem merch! in...,so cut missed out on all the tce extreme merch...


In [10]:
all_train_data[all_train_data.isnull().any(axis=1)]

,id,text,cat,text_no_punc,text_no_numerals,text_no_sw,text_porter_stemmed,text_lancaster_stemmed,text_lemmatized


In [11]:
all_test_data[all_test_data.isnull().any(axis=1)]

,id,text,cat,text_no_punc,text_no_numerals,text_no_sw,text_porter_stemmed,text_lancaster_stemmed,text_lemmatized


load train targets, binarize, drop nuetrals

In [12]:
train_target = pd.read_csv(input_path + 'train_results.csv')
train_target['target'].value_counts()

positive    520436
negative    519803
neutral         84
Name: target, dtype: int64

In [13]:
train_target.head()

,id,target
0,0,positive
1,1,negative
2,2,positive
3,3,positive
4,4,negative


In [14]:
train_target['target'] = [0 if t == 'negative' else 2 if t == 'positive' else 2 for t in train_target['target'].values]

In [15]:
all_train_data['target'] = train_target['target']

drop nuetral target rows to keep it binary

In [16]:
# all_train_data = all_train_data[all_train_data['target'] != 'neutral']
# all_train_data['target'] = [1 if t=='positive' else 0 for t in all_train_data['target'].values]
all_train_data['target'].value_counts()


2    520520
0    519803
Name: target, dtype: int64

### prepare bag of words<br>
we need a different model for each text format

Naive Bayes doesn't have any hyperparameters, so we'll test different versions of the text processing with different versions of naive bayes and corpos generation

the following parameters will be tested<br>
1. size of corpus vocab
2. corpus ngram varieties
3. bernoulli, multinomial or gaussian naive bayes
4. stemming/lemmatizing methods

In [17]:
vocab_sizes = [128, 1024, 8096, 16384]
ngrams = [(1, 1), (2, 2), (1, 2), (1, 3), (1, 4)]
models = [BernoulliNB(), GaussianNB(), MultinomialNB()]
text_columns = ['text_no_numerals', 'text_no_sw', 'text_porter_stemmed', 'text_lancaster_stemmed', 'text_lemmatized']
threshold = 0.5

# shuffle train data
all_train_data = all_train_data.sample(len(all_train_data))

# get data and target
train_X = all_train_data.iloc[:, :-1]
train_y  = all_train_data.iloc[:, -1]


# get 5000 data points for first round of training
train_v1 = train_X[:10_000]
targets_v1 = train_y[:10_000]

In [18]:
targets_v1.head()

913344    0
626605    2
565148    2
209687    2
698022    2
Name: target, dtype: int64

In [19]:
def trainModels(df, targets, vocab_sizes, models, text_columns, ngrams, threshold):

    param_tracker = []
    counter = 1
    for size in tqdm(vocab_sizes):

        for model in models:

            for col in text_columns:


                for ng in tqdm(ngrams):

                    counter+=1
                    if counter % 10==0:
                        print(f'{counter}')
                    vectorizer = CountVectorizer(max_features=size, ngram_range=ng)

                    BOW = vectorizer.fit_transform(df[col])
                    BOW_array = BOW.toarray()

                    X_train, X_valid, y_train, y_valid = train_test_split(BOW_array, targets, test_size=0.2, random_state=42)


                    model.fit(X_train, y_train)


                    prob_predictions = model.predict_proba(X_valid)

                    # gets index of 1 column in prob_predictions
                    pos_predictions = [pred[list(model.classes_).index(2)] for pred in prob_predictions]

                    num_correct_pred = 0
                    for pred, actual in zip(pos_predictions, y_valid):
                        if pred >= threshold:
                            binary_pred = 2
                        else:
                            binary_pred = 0

                        if binary_pred==actual:
                            num_correct_pred+=1
                    accuracy = num_correct_pred / len(y_valid)
                    print(f'accuracy: {accuracy}')
                    print('\n\n')

                    param_tracker.append((size, str(model), col, ng, accuracy))
                    
    param_df = pd.DataFrame(param_tracker, columns = ['vocab_size', 'model_type', 'text_column', 'ngram', 'accuracy_round1'])
    param_df = param_df.sort_values(by='accuracy_round1', ascending=False)
    return param_df

                    
                
    # for vocab size we'll test

In [20]:
param_df = trainModels(train_v1, targets_v1, vocab_sizes, models, text_columns, ngrams, threshold)

 20%|██████████████                                                        | 1/5 [00:00<00:00,  4.42it/s]

accuracy: 0.6915






 40%|████████████████████████████                                          | 2/5 [00:00<00:00,  3.65it/s]

accuracy: 0.587






 60%|██████████████████████████████████████████                            | 3/5 [00:00<00:00,  3.09it/s]

...

accuracy: 0.7415



300



 80%|████████████████████████████████████████████████████████              | 4/5 [00:24<00:06,  6.27s/it]

accuracy: 0.747






100%|█████████████████████████████████████████████████████████████████████| 4/4 [13:47<00:00, 206.85s/it]

accuracy: 0.7395





In [21]:
param_df

,vocab_size,model_type,text_column,ngram,accuracy_round1
169,8096,BernoulliNB(),text_lancaster_stemmed,"(1, 4)",0.7505
164,8096,BernoulliNB(),text_porter_stemmed,"(1, 4)",0.7505
168,8096,BernoulliNB(),text_lancaster_stemmed,"(1, 3)",0.7500
174,8096,BernoulliNB(),text_lemmatized,"(1, 4)",0.7500
243,16384,BernoulliNB(),text_lancaster_stemmed,"(1, 3)",0.7500
...,...,...,...,...,...
260,16384,GaussianNB(),text_porter_stemmed,"(1, 1)",0.5515
131,1024,MultinomialNB(),text_no_sw,"(2, 2)",0.5505
31,128,GaussianNB(),text_no_sw,"(2, 2)",0.5425
6,128,BernoulliNB(),text_no_sw,"(2, 2)",0.5410


In [22]:
param_df.to_csv('data/output/naive_bayes/naive_bayes_params_round1.csv', index=False)

In [25]:
param_df = pd.read_csv('data/output/naive_bayes/naive_bayes_params_round1.csv')

# ngrams get saved as string, convert back to int tuple
param_df['ngram'] = [(int(ng[1]), int(ng[4])) for ng in param_df['ngram'].values]

param_df = param_df.replace({np.nan: None})

In [26]:
top_params = param_df[:10]

based on these results, we'll train the full dataset on these 10 combinations

In [27]:
num_points = 100_000
num_train = int(num_points*0.8)

In [28]:

model_accuracy = []

for i, param_set in enumerate(tqdm(top_params.values)):
    vocab_size, model_type, text_column, ngram, *_ = param_set
    print(vocab_size, model_type, text_column, ngram,)
    
    print('splitting train-validation data')
    
    # train is all training data with targets included!
    # sample to get smaller amount of data to deal with RAM constraints
    train_sample = all_train_data.sample(num_points)

    # get data and target
    sample_train_X = train_sample.iloc[:, :-1]
    sample_train_y  = train_sample.iloc[:, -1]



    if vocab_size:
        vocab_size = int(vocab_size)
    
    if model_type == 'MultinomialNB()':
        model = MultinomialNB()
    elif model_type == 'BernoulliNB()':
        model = BernoulliNB()
    elif model_type == 'GaussianNB()':
        model = GaussianNB()
    else:
        print('warning, unrecognized model!')
        print(i, param_set)
        
    print('vectorizing')
    vectorizer = CountVectorizer(max_features=vocab_size, ngram_range=ngram)

    print('creating BOW')
    
    BOW = vectorizer.fit_transform(sample_train_X[text_column])
    # BOW_array = BOW.toarray()
    

    # all sample_train_X and sample_train_y are the X, y sample from all of the training data, 
    #+ sample size determined by num_points
    X_train, X_valid, y_train, y_valid = train_test_split(BOW, sample_train_y, test_size=0.2, random_state=42)
    
    print('fitting model')
    model.fit(X_train, y_train)

    print('getting predictions')
    prob_predictions = model.predict_proba(X_valid)

    # gets index of 1 column in prob_predictions
    pos_predictions = [pred[list(model.classes_).index(2)] for pred in prob_predictions]
    
    
    # get accuracy
    num_correct_pred = 0
    for pred, actual in zip(pos_predictions, y_valid):
        if pred >= threshold:
            binary_pred = 2
        else:
            binary_pred = 0

        if binary_pred==actual:
            num_correct_pred+=1
    accuracy = num_correct_pred / len(y_valid)
    print(accuracy)
    print('\n')
    
    model_accuracy.append(accuracy)
    
    

  0%|                                                                             | 0/10 [00:00<?, ?it/s]

8096 BernoulliNB() text_lancaster_stemmed (1, 4)
splitting train-validation data
vectorizing
creating BOW


 10%|██████▉                                                              | 1/10 [00:11<01:44, 11.66s/it]

fitting model
getting predictions
0.76435


8096 BernoulliNB() text_porter_stemmed (1, 4)
splitting train-validation data
vectorizing
creating BOW


 20%|█████████████▊                                                       | 2/10 [00:23<01:31, 11.49s/it]

...

fitting model
getting predictions
0.7642


8096 BernoulliNB() text_porter_stemmed (1, 2)
splitting train-validation data
vectorizing
creating BOW


 90%|██████████████████████████████████████████████████████████████       | 9/10 [01:03<00:04,  4.77s/it]

fitting model
getting predictions
0.76475


16384 MultinomialNB() text_lancaster_stemmed (1, 2)
splitting train-validation data
vectorizing
creating BOW


100%|████████████████████████████████████████████████████████████████████| 10/10 [01:07<00:00,  6.71s/it]

fitting model
getting predictions
0.77905




In [29]:
top_params['accuracy_round2'] = model_accuracy

/Users/Tanner1/Desktop/MILA/Fall2022/IFT6390 - ML/kaggle/ift6390-kaggle-2/env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [30]:
top_params = top_params.sort_values('accuracy_round2', ascending=False)

In [31]:
top_params

,vocab_size,model_type,text_column,ngram,accuracy_round1,accuracy_round2
9,16384,MultinomialNB(),text_lancaster_stemmed,"(1, 2)",0.7480,0.77905
4,16384,BernoulliNB(),text_lancaster_stemmed,"(1, 3)",0.7500,0.76835
5,16384,BernoulliNB(),text_lancaster_stemmed,"(1, 2)",0.7495,0.76715
6,8096,BernoulliNB(),text_lancaster_stemmed,"(1, 2)",0.7495,0.76585
8,8096,BernoulliNB(),text_porter_stemmed,"(1, 2)",0.7480,0.76475
0,8096,BernoulliNB(),text_lancaster_stemmed,"(1, 4)",0.7505,0.76435
7,8096,BernoulliNB(),text_lemmatized,"(1, 2)",0.7485,0.76420
3,8096,BernoulliNB(),text_lemmatized,"(1, 4)",0.7500,0.76320
1,8096,BernoulliNB(),text_porter_stemmed,"(1, 4)",0.7505,0.76220
2,8096,BernoulliNB(),text_lancaster_stemmed,"(1, 3)",0.7500,0.75555


In [33]:
top_params.to_csv('data/output/naive_bayes/naive_bayes_params_round2.csv', index=False)

## train final model with best parameters

In [34]:
all_train_data.head(1)

,id,text,cat,text_no_punc,text_no_numerals,text_no_sw,text_porter_stemmed,text_lancaster_stemmed,text_lemmatized,target
913344,913344,wished... but i gues it aint gonna happen. ily,train,wished but i gues it aint gonna happen ily,wished but i gues it aint gonna happen ily,wished gues aint gonna happen ily,wish but i gue it aint gonna happen ili,wish but i gue it aint gonn hap ily,wished but i gues it aint gonna happen ily,0


In [ ]:
# # 8096	BernoulliNB()	text_porter_stemmed	(1, 2)	

# all_train_data = all_train_data.sample(len(all_train_data))


vectorizer = CountVectorizer(max_features=16384, ngram_range=(1, 2))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(all_train_data['text_lancaster_stemmed'], 
                                                    all_train_data['target'], 
                                                    test_size=0.2, 
                                                    random_state=42)


In [ ]:
train_BOW = vectorizer.fit_transform(X_train)



In [ ]:
test_BOW = vectorizer.transform(X_test)
# test_BOW_array = test_BOW.toarray()

In [ ]:
model = BernoulliNB()
model.fit(train_BOW, y_train, test_BOW, y_test)

MultinomialNB()

In [ ]:
model.classes_

array([0, 2])

In [ ]:
predictions = model.predict(test_BOW_array)


In [ ]:
pdf = pd.DataFrame(predictions).reset_index()
pdf.columns  = ['id', 'target']


In [ ]:
pdf.to_csv('data/output/first_pred.csv', index=False)

code from https://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html

## plot ROC curve